In [86]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import json

In [30]:
def read_links(page_num, model):
    page = f'https://auto.ru/moskva/cars/{model}/used/?year_to=2020&page={page_num}&output_type=table'
    response = requests.get(page)
    soup = BeautifulSoup(response.content.decode("utf-8"), 'html.parser')
    class_ = "Link ListingItemTitle__link"
    page_link = soup.find_all('a', class_=class_)
    hrefs=[]
    for a in page_link:
        hrefs.append(a.get('href', None))
    return hrefs


In [41]:
def links_count(model):
    page = f'https://auto.ru/moskva/cars/{model}/used/?year_to=2020&output_type=table'
    response = requests.get(page)
    soup = BeautifulSoup(response.content.decode("utf-8"), 'html.parser')
    class_ = "Button Button_color_whiteHoverBlue Button_size_s Button_type_link Button_width_default ListingPagination-module__page"
    page_links = soup.find_all('a', class_=class_)
    return int(page_links[-1].find('span', class_="Button__text").text)


In [42]:
links_count('skoda')

62

In [43]:
models = ['skoda', 'audi', 'honda', 'volvo', 'bmw', 'nissan', 'infiniti',
          'mercedes', 'toyota', 'lexus', 'volkswagen', 'mitsubishi']
models_dict = dict()
for model in models:
    models_dict[model] = links_count(model)
models_dict

{'skoda': 62,
 'audi': 74,
 'honda': 26,
 'volvo': 33,
 'bmw': 99,
 'nissan': 89,
 'infiniti': 16,
 'mercedes': 99,
 'toyota': 97,
 'lexus': 29,
 'volkswagen': 99,
 'mitsubishi': 62}

In [50]:
n = 0
for model in models_dict.keys():
    with open(f'links_{model}.csv', 'w') as output:
        print(f'Pages for {model} count = {models_dict[model]}')
        for i in range(models_dict[model]):
            print('Page =', i + 1, 'model =', model)
            hrefs = read_links(i + 1, model)
            for href in hrefs:
                output.write(href)
                output.write('\n')

Pages for bmw count = 99
Page = 1 model = bmw
Page = 2 model = bmw
Page = 3 model = bmw
Page = 4 model = bmw
Page = 5 model = bmw
Page = 6 model = bmw
Page = 7 model = bmw
Page = 8 model = bmw
Page = 9 model = bmw
Page = 10 model = bmw
Page = 11 model = bmw
Page = 12 model = bmw
Page = 13 model = bmw
Page = 14 model = bmw
Page = 15 model = bmw
Page = 16 model = bmw
Page = 17 model = bmw
Page = 18 model = bmw
Page = 19 model = bmw
Page = 20 model = bmw
Page = 21 model = bmw
Page = 22 model = bmw
Page = 23 model = bmw
Page = 24 model = bmw
Page = 25 model = bmw
Page = 26 model = bmw
Page = 27 model = bmw
Page = 28 model = bmw
Page = 29 model = bmw
Page = 30 model = bmw
Page = 31 model = bmw
Page = 32 model = bmw
Page = 33 model = bmw
Page = 34 model = bmw
Page = 35 model = bmw
Page = 36 model = bmw
Page = 37 model = bmw
Page = 38 model = bmw
Page = 39 model = bmw
Page = 40 model = bmw
Page = 41 model = bmw
Page = 42 model = bmw
Page = 43 model = bmw
Page = 44 model = bmw
Page = 45 model 

Page = 25 model = toyota
Page = 26 model = toyota
Page = 27 model = toyota
Page = 28 model = toyota
Page = 29 model = toyota
Page = 30 model = toyota
Page = 31 model = toyota
Page = 32 model = toyota
Page = 33 model = toyota
Page = 34 model = toyota
Page = 35 model = toyota
Page = 36 model = toyota
Page = 37 model = toyota
Page = 38 model = toyota
Page = 39 model = toyota
Page = 40 model = toyota
Page = 41 model = toyota
Page = 42 model = toyota
Page = 43 model = toyota
Page = 44 model = toyota
Page = 45 model = toyota
Page = 46 model = toyota
Page = 47 model = toyota
Page = 48 model = toyota
Page = 49 model = toyota
Page = 50 model = toyota
Page = 51 model = toyota
Page = 52 model = toyota
Page = 53 model = toyota
Page = 54 model = toyota
Page = 55 model = toyota
Page = 56 model = toyota
Page = 57 model = toyota
Page = 58 model = toyota
Page = 59 model = toyota
Page = 60 model = toyota
Page = 61 model = toyota
Page = 62 model = toyota
Page = 63 model = toyota
Page = 64 model = toyota


In [167]:
if os.path.exists("pages"):
    os.remove("pages/*")
else:
    os.mkdir('pages')

In [169]:
!wget -i links.csv -P pages

--2021-08-26 19:27:03--  https://auto.ru/cars/used/sale/citroen/c4/1103990932-09c5b61a/
Resolving auto.ru (auto.ru)... 213.180.204.188
Connecting to auto.ru (auto.ru)|213.180.204.188|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 388922 (380K) [text/html]
Saving to: ‘pages/index.html’

index.html          100%[===================>] 379,81K  1,68MB/s    in 0,2s    

2021-08-26 19:27:04 (1,68 MB/s) - ‘pages/index.html’ saved [388922/388922]

--2021-08-26 19:27:04--  https://auto.ru/cars/used/sale/genesis/g70/1104268899-413b9a3a/
Reusing existing connection to auto.ru:443.
HTTP request sent, awaiting response... 200 OK
Length: 446461 (436K) [text/html]
Saving to: ‘pages/index.html.1’

index.html.1        100%[===================>] 436,00K  1,78MB/s    in 0,2s    

2021-08-26 19:27:05 (1,78 MB/s) - ‘pages/index.html.1’ saved [446461/446461]

--2021-08-26 19:27:05--  https://auto.ru/cars/used/sale/genesis/gv80/1104788625-6af84c82/
Reusing existing connection to au

HTTP request sent, awaiting response... 200 OK
Length: 411782 (402K) [text/html]
Saving to: ‘pages/index.html.20’

index.html.20       100%[===================>] 402,13K  2,19MB/s    in 0,2s    

2021-08-26 19:27:18 (2,19 MB/s) - ‘pages/index.html.20’ saved [411782/411782]

--2021-08-26 19:27:18--  https://auto.ru/cars/used/sale/toyota/camry/1103729206-17bbac17/
Reusing existing connection to auto.ru:443.
HTTP request sent, awaiting response... 200 OK
Length: 413221 (404K) [text/html]
Saving to: ‘pages/index.html.21’

index.html.21       100%[===================>] 403,54K  1,99MB/s    in 0,2s    

2021-08-26 19:27:19 (1,99 MB/s) - ‘pages/index.html.21’ saved [413221/413221]

--2021-08-26 19:27:19--  https://auto.ru/cars/used/sale/lexus/lx/1103420931-574e4947/
Reusing existing connection to auto.ru:443.
HTTP request sent, awaiting response... 200 OK
Length: 424831 (415K) [text/html]
Saving to: ‘pages/index.html.22’

index.html.22       100%[===================>] 414,87K  2,58MB/s    in 

HTTP request sent, awaiting response... 200 OK
Length: 411595 (402K) [text/html]
Saving to: ‘pages/index.html.60’

index.html.60       100%[===================>] 401,95K  2,53MB/s    in 0,2s    

2021-08-26 19:27:46 (2,53 MB/s) - ‘pages/index.html.60’ saved [411595/411595]

--2021-08-26 19:27:46--  https://auto.ru/cars/used/sale/mercedes/cls_klasse/1104767956-00faf4d4/
Reusing existing connection to auto.ru:443.
HTTP request sent, awaiting response... 200 OK
Length: 449334 (439K) [text/html]
Saving to: ‘pages/index.html.61’

index.html.61       100%[===================>] 438,80K  1,05MB/s    in 0,4s    

2021-08-26 19:27:47 (1,05 MB/s) - ‘pages/index.html.61’ saved [449334/449334]

--2021-08-26 19:27:47--  https://auto.ru/cars/used/sale/lexus/rx/1104677063-bdaa1f1f/
Reusing existing connection to auto.ru:443.
HTTP request sent, awaiting response... 200 OK
Length: 426607 (417K) [text/html]
Saving to: ‘pages/index.html.62’

index.html.62       100%[===================>] 416,61K  2,09MB/s

HTTP request sent, awaiting response... 200 OK
Length: 398321 (389K) [text/html]
Saving to: ‘pages/index.html.100’

index.html.100      100%[===================>] 388,99K  1,96MB/s    in 0,2s    

2021-08-26 19:28:16 (1,96 MB/s) - ‘pages/index.html.100’ saved [398321/398321]

--2021-08-26 19:28:16--  https://auto.ru/cars/used/sale/lexus/gx/1103720723-d9f4f1cb/
Reusing existing connection to auto.ru:443.
HTTP request sent, awaiting response... 200 OK
Length: 432652 (423K) [text/html]
Saving to: ‘pages/index.html.101’

index.html.101      100%[===================>] 422,51K  2,07MB/s    in 0,2s    

2021-08-26 19:28:17 (2,07 MB/s) - ‘pages/index.html.101’ saved [432652/432652]

--2021-08-26 19:28:17--  https://auto.ru/cars/used/sale/toyota/hilux/1103957837-729e8992/
Reusing existing connection to auto.ru:443.
HTTP request sent, awaiting response... 200 OK
Length: 396608 (387K) [text/html]
Saving to: ‘pages/index.html.102’

index.html.102      100%[===================>] 387,31K  2,14MB/s  

In [246]:
def parse_row(parent, class_):
    row = parent.find('li', class_=class_)
    if row is not None:
        cell = row.find_all('span', class_="CardInfoRow__cell")
        if cell is not None and len(cell) > 1:
            return cell[1].text.replace('\xa0', '')
    return None    

def parse_page(filename):
    car = dict()
    with open(filename, 'r') as page:
        soup = BeautifulSoup(page.read(), 'html.parser')
        head = soup.find('head')
        canonical = head.find(attrs={"rel" : "canonical"})
        car['car_url'] = canonical['href']
        car_json = json.loads(soup.find('script', attrs={"id" : "initial-state"}).contents[0])
        car['brand'] = car_json['card']['vehicle_info']['mark_info']['code'].upper()
        
        page_card = soup.find('div', class_='PageCard')   
        card_info = page_card.find('ul', class_='CardInfo')
        car['color'] = parse_row(card_info, class_='CardInfoRow CardInfoRow_color')
        car['steering_wheel'] = car_json['card']['vehicle_info']['steering_wheel']

        car['body_type'] = car_json['card']['vehicle_info']['configuration']['body_type']
        car['description'] = car_json['card'].get('description', None)
        car['equipment_dict'] = car_json['card']['vehicle_info']['equipment']
        car['mileage'] = car_json['card']['state']['mileage']
        car['modelDate'] = car_json['card']['vehicle_info']['super_gen']['year_from']
        car['model_info'] = car_json['card']['vehicle_info']['model_info']
        car['numberOfDoors'] = car_json['card']['vehicle_info']['configuration']['doors_count']
        car['productionDate'] = car_json['card']['documents']['year']

        car['owners'] = car_json['card']['documents']['owners_number']
        car['pts_original'] = car_json['card']['documents'].get('pts_original', None)
        
        car['tech_param'] = car_json['card']['vehicle_info']['tech_param']

        car['price_info'] = car_json['card']['price_info']
        return car

In [247]:
# ! wget https://auto.ru/cars/used/sale/skoda/superb/1100658222-7ac3def5/

In [248]:
parse_page(f'index.html')

{'car_url': 'https://auto.ru/cars/used/sale/skoda/superb/1100658222-7ac3def5/',
 'brand': 'SKODA',
 'color': 'серый',
 'steering_wheel': 'LEFT',
 'body_type': 'LIFTBACK',
 'description': 'Все автомобили, представленные в продаже, проходят тщательную проверку по более 40 параметрам. Предоставляем гарантию юридической чистоты, а так же год технической гарантии на двигатель и КПП. Бесплатный тест-драйв. Возможно оформление автомобиля в кредит!\n\nПреимущества автокредитования:\n— Первоначальный взнос от 0%;\n— Более 30 кредитных программ;\n— Процентная ставка от 4.9% годовых;\n— Срок кредита от 6 месяцев до 7 лет;\n— Оформление кредита по двум документам;\n— Досрочное погашение без штрафов и комиссий;\n— Сумма кредита до 2 млн рублей;\n— Оформление КАСКО – по желанию;\n— Без справок и поручителей.\n\nСотрудничаем с 12 аккредитованными и сертифицированными банками РФ, среднее время ожидания решения банка 20–30 минут.\n\nПолучите дополнительную скидку до 50 000 рублей при покупке автомобиля

In [253]:
count = 10
with open('/Volumes/Z Slim/train.json', 'w') as output:
    tree = os.walk("/Volumes/Z Slim/auto.ru")
    for folder, subfolder, filenames in tree:
        print(folder, len(filenames))
        for filename in filenames:
            if '.html' in filename:
                try:
                    page = parse_page(f'{folder}/{filename}')
                    json.dump(page, output, ensure_ascii=False)
                    output.write('\n')
                except KeyError as e:
                    print(e)
#                if count == 0:
#                    break
#                count -= 1


/Volumes/Z Slim/auto.ru 14
/Volumes/Z Slim/auto.ru/audi 2710
/Volumes/Z Slim/auto.ru/bmv 3663
/Volumes/Z Slim/auto.ru/honda 957
/Volumes/Z Slim/auto.ru/infiniti 587
/Volumes/Z Slim/auto.ru/lexus 1049
'vehicle_info'
/Volumes/Z Slim/auto.ru/mercedes 3662
/Volumes/Z Slim/auto.ru/mitsubishi 2274
/Volumes/Z Slim/auto.ru/nissan 3255
'owners_number'
/Volumes/Z Slim/auto.ru/skoda 2273
/Volumes/Z Slim/auto.ru/toyota 3554
/Volumes/Z Slim/auto.ru/volkswagen 3662
/Volumes/Z Slim/auto.ru/volvo 1213
